In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
dbutils.fs.ls("/Volumes/azuredatabrickss/default/sales")


[FileInfo(path='dbfs:/Volumes/azuredatabrickss/default/sales/sales_data_sample.csv', name='sales_data_sample.csv', size=527958, modificationTime=1721797252000)]

In [0]:
df1 = spark.read.format("csv").option("header", "true").option("inferschema","true").load("dbfs:/Volumes/azuredatabrickss/default/sales/sales_data_sample.csv")

In [0]:
df1.show(5)

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped| 

Customer Sales Contribuition

In [0]:
df1.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [0]:
df1.createOrReplaceTempView("productsales")

In [0]:
spark.sql('select customername,sum(sales) as sum_sales,sum(sales)/(select sum(sales) from productsales) as contribuition_percentage from productsales group by customername order by sum_sales desc').show(10)

+--------------------+------------------+------------------------+
|        customername|         sum_sales|contribuition_percentage|
+--------------------+------------------+------------------------+
|Euro Shopping Cha...| 912294.1100000002|     0.09093270803095634|
|Mini Gifts Distri...|         654858.06|     0.06527282826773756|
|Australian Collec...|200995.40999999997|    0.020034171801341955|
|  Muscle Machine Inc|197736.93999999997|    0.019709384544809486|
|   La Rochelle Gifts|          180124.9|    0.017953908461389938|
|Dragon Souveniers...|172989.68000000008|    0.017242707029872822|
|   Land of Toys Inc.|164069.44000000003|    0.016353584135627607|
|The Sharp Gifts W...|160010.26999999996|     0.01594898728861078|
|      AV Stores, Co.|157807.80999999997|     0.01572945758877543|
|Anna's Decoration...|153996.13000000003|    0.015349529251249026|
+--------------------+------------------+------------------------+
only showing top 10 rows



In [0]:
#sesonal sales trend

In [0]:

%sql
select year_id,qtr_id, sum(sales) as sum_sales,sum(sales)/(select sum(sales)from productsales) as contribuition_percentage from productsales group by qtr_id,year_id order by qtr_id 


year_id,qtr_id,sum_sales,contribuition_percentage
2003,1,445094.6900000002,0.04436471204653401
2005,1,1071992.3600000003,0.10685059479699573
2004,1,833730.6800000005,0.08310191600479665
2003,2,562365.22,0.05605362546627043
2005,2,719494.3500000001,0.07171543578032386
2004,2,766260.7299999996,0.07637686407585972
2003,3,649514.5399999999,0.06474021412643001
2004,3,1109396.2700000005,0.11057882102356445
2004,4,2014774.9199999995,0.20082223215104755
2003,4,1860005.0899999987,0.18539558452817645


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select month_id,
case when qtr_id==1 then "Winter"

 when qtr_id==2 then "Summer"
 when qtr_id==3 then "Spring"
 when qtr_id==4 then "Fall" end as season, sum(sales) as sum_sales from productsales group by month_id,qtr_id order by month_id ASC

month_id,season,sum_sales
1,Winter,785874.4400000008
10,Fall,1121215.2199999997
11,Fall,2118885.67
12,Fall,634679.1199999998
2,Winter,810441.9
3,Winter,754501.3900000001
4,Summer,669390.9600000003
5,Summer,923972.56
6,Summer,454756.77999999985
7,Spring,514875.9700000001


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select sum(sales) as Total_Sales from productsales


Total_Sales
1.003262885000001E7


In [0]:
%sql
select count(customername) as Total_Customers  from productsales

Total_Customers
2823


In [0]:
%sql
select priceeach,quantityordered,sales,msrp from productsales group by msrp

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2020487724883787>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2020487724883787>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0IHByaWNlYWNoLHF1YW50aXR5b3JkZXJlZCxtc3JwIGZyb20gcHJvZHVjdHNhbGVz").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )

In [0]:
%sql
SELECT priceeach, quantityordered, sales, msrp
FROM productsales
GROUP BY msrp, priceeach, quantityordered;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2020487724883788>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2020487724883788>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIHByaWNlZWFjaCwgcXVhbnRpdHlvcmRlcmVkLCBzYWxlcywgbXNycApGUk9NIHByb2R1Y3RzYWxlcwpHUk9VUCBCWSBtc3JwLCBwcmljZWVhY2gsIHF1YW50aXR5b3JkZXJl").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name,

In [0]:
%sql
select sum(sales-(priceeach*quantityordered)) as Total_Profit from productsales

Total_Profit
1741742.059999999


In [0]:
#quantity  sold per prodcut

In [0]:
%sql
select productline,sum(quantityordered) as Products_Sold from productsales group by productline order by productline

productline,Products_Sold
Classic Cars,33992
Motorcycles,11663
Planes,10727
Ships,8127
Trains,2712
Trucks and Buses,10777
Vintage Cars,21069


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select month_id,sum(quantityordered) as Products_Sold_per_Month from productsales group by month_id order by month_id

month_id,Products_Sold_per_Month
1,7997.0
10,10998.0
11,20857.0
12,6293.0
2,7903.0
3,7585.0
4,6704.0
5,8992.0
6,4620.0
7,4899.0


Databricks visualization. Run in Databricks to view.

In [0]:
spark.sql("select month_id,sum(quantityordered) as Products_Sold_per_Month from productsales group by month_id order by month_id").display()

month_id,Products_Sold_per_Month
1,7997.0
10,10998.0
11,20857.0
12,6293.0
2,7903.0
3,7585.0
4,6704.0
5,8992.0
6,4620.0
7,4899.0


In [0]:
%sql
select customername as Customer_Name,sum(sales) as sum_sales,sum(sales)/(select sum(sales)from prodcutsales) as Contribution_Pcnt from prodcutsales group by CUSTOMERNAME order by sum_sales DESC limit 10

Customer_Name,sum_sales,Contribution_Pcnt
Euro Shopping Channel,912294.1100000002,0.09093270803095634
Mini Gifts Distributors Ltd.,654858.06,0.06527282826773756
"Australian Collectors, Co.",200995.40999999997,0.020034171801341955
Muscle Machine Inc,197736.93999999997,0.019709384544809486
La Rochelle Gifts,180124.9,0.017953908461389938
"Dragon Souveniers, Ltd.",172989.68000000008,0.017242707029872822
Land of Toys Inc.,164069.44000000003,0.016353584135627607
The Sharp Gifts Warehouse,160010.26999999996,0.01594898728861078
"AV Stores, Co.",157807.80999999997,0.01572945758877543
"Anna's Decorations, Ltd",153996.13000000003,0.015349529251249026


In [0]:
#top 3 highest selling product in each quater based on Total Sales

In [0]:
%sql

select qtr_id, productline, rank
from (
  select qtr_id,productline,
         rank() over (partition by qtr_id order by sum(sales) desc) as rank
  from productsales
  group by qtr_id, productline
) as ranked_tb
where rank <= 3
order by qtr_id, rank;



qtr_id,productline,rank
1,Classic Cars,1
1,Vintage Cars,2
1,Motorcycles,3
2,Classic Cars,1
2,Vintage Cars,2
2,Motorcycles,3
3,Classic Cars,1
3,Vintage Cars,2
3,Motorcycles,3
4,Classic Cars,1


In [0]:
#cateogorize productline as 'Hot','cold'  based on their orderquantity in each quARTER USING CASE WHEN STATEMENMT

In [0]:
%sql 

select qtr_id,min(rank) from (
select qtr_id, productline,rank
from (
  select qtr_id,productline,
         rank() over (partition by qtr_id order by sum(sales) desc) as rank
  from productsales
  group by qtr_id, productline
) as ranked_tb
where rank <= 3 
order by qtr_id, rank) group by qtr_id

qtr_id,min(rank)
1,1
2,1
3,1
4,1


In [0]:
%sql

select productline,qtr_id,min(sumo),max(sumo) from (
select productline,qtr_id,sum(quantityordered) as sumo from productsales group by productline,qtr_id order by qtr_id) group by productline,qtr_id



productline,qtr_id,min(sumo),max(sumo)
Ships,1,2163.0,2163.0
Planes,1,2625.0,2625.0
Vintage Cars,1,5481.0,5481.0
Trains,1,693.0,693.0
Classic Cars,1,7784.0,7784.0
Motorcycles,1,2671.0,2671.0
Trucks and Buses,1,2068.0,2068.0
Classic Cars,2,6447.0,6447.0
Planes,2,2773.0,2773.0
Trucks and Buses,2,2286.0,2286.0


In [0]:
%sql
select country,rank() over(partition by productline order by sum(quantityordered) desc ) as rank from productsales group by country order by rank

country,rank
USA,1
USA,1
USA,1
USA,1
USA,1
USA,1
USA,1
Spain,2
France,2
France,2


In [0]:
WITH ProductLineSales AS (
  SELECT country, productline, SUM(quantityordered) AS total_quantity
  FROM productsales
  WHERE orderdate >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
  GROUP BY country, productline
)
SELECT country,
       productline,
       RANK() OVER (PARTITION BY country ORDER BY total_quantity DESC) AS rank
FROM ProductLineSales
ORDER BY country, rank;


In [0]:
#calculate the rank of products.show the rank of products in each quater

In [0]:
%sql
WITH RankedProducts AS (
  SELECT
    qtr_id,
    productline,
    RANK() OVER (PARTITION BY qtr_id ORDER BY SUM(quantityordered) DESC) AS quantity_rank,
    RANK() OVER (PARTITION BY qtr_id ORDER BY SUM(quantityordered) ASC) AS quantity_rank_asc
  FROM productsales
  GROUP BY qtr_id, productline
)
SELECT
  DISTINCT qtr_id,
  FIRST_VALUE(productline) OVER (PARTITION BY qtr_id ORDER BY quantity_rank) AS hot_product,
  FIRST_VALUE(productline) OVER (PARTITION BY qtr_id ORDER BY quantity_rank_asc) AS cold_product
FROM RankedProducts
WHERE quantity_rank = 1 OR quantity_rank_asc = 1;


qtr_id,hot_product,cold_product
1,Classic Cars,Trains
2,Classic Cars,Trains
3,Classic Cars,Trains
4,Classic Cars,Trains


In [0]:
###6) Identify the product with outliers sales in each quarter.[Q3 + 1.5(q3-q1)]

In [0]:
%sql
WITH Quartiles AS (
  SELECT
    QTR_ID,
    PRODUCTCODE,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY SALES) AS Q1,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY SALES) AS Q3
    FROM productsales
      GROUP BY QTR_ID,PRODUCTCODE),
OutlierBounds AS(

  SELECT
    QTR_ID,
    PRODUCTCODE,
    Q1,
    Q3,
    (Q3 + 1.5 *(Q3-Q1)) AS UpperBound,
    (Q1-1.5 * (Q3-Q1)) AS LowerBound
  FROM Quartiles
)
SELECT
    s.QTR_ID,
    s.PRODUCTCODE,
    s.SALES,
    ob.UpperBound,
    ob.LowerBound
FROM productsales s
JOIN OutlierBounds ob
ON s.QTR_ID = ob.QTR_ID AND s.PRODUCTCODE = ob.PRODUCTCODE
WHERE s.SALES > ob.UpperBound OR s.SALES < ob.LowerBound


QTR_ID,PRODUCTCODE,SALES,UpperBound,LowerBound
1,S18_2625,6981.0,3140.1462500000007,201.41624999999976
1,S24_3969,4086.29,1929.1987499999998,1345.9687500000002
3,S18_4027,7140.76,5999.72,2466.5999999999995
2,S24_4258,2208.92,4244.3675,2497.5874999999996
2,S10_1949,12001.0,10021.09,3779.89
2,S24_1046,3880.2,3592.515,573.635
4,S32_3522,5669.64,4646.3775000000005,665.8374999999996
3,S72_1253,948.99,2600.0912500000004,1068.4012499999997
1,S700_2047,3910.32,3782.178750000001,2002.5887499999992
2,S24_3151,6107.4,5127.225,1809.5849999999994


In [0]:
### COMPUTE mOVING AVERAGE FOR EACH PRODUCT

In [0]:

%sql
SELECT
  YEAR_ID,
  MONTH_ID,
  PRODUCTCODE,
  SALES,
  AVG(SALES) OVER (PARTITION BY PRODUCTCODE ORDER BY YEAR_ID,MONTH_ID ROWS BETWEEN 3 PRECEDING AND CURRENT ROW)
  AS MOVING_AVG_3_MONTHS
FROM productsales
ORDER BY PRODUCTCODE,YEAR_ID,MONTH_ID

YEAR_ID,MONTH_ID,PRODUCTCODE,SALES,MOVING_AVG_3_MONTHS
2003,2,S10_1678,2871.0,2871.0
2003,5,S10_1678,2765.9,2818.45
2003,7,S10_1678,3884.34,3173.7466666666664
2003,8,S10_1678,3746.7,3316.9849999999997
2003,10,S10_1678,5205.27,3900.5525
2003,10,S10_1678,3479.76,4079.0175000000004
2003,11,S10_1678,2497.77,3732.3750000000005
2003,11,S10_1678,5512.32,4173.780000000001
2003,12,S10_1678,2168.54,3414.5975
2004,1,S10_1678,4708.44,3721.7675


In [0]:
### Rolling 3-Month Sales Average for Each Product

In [0]:
%sql
SELECT
  PRODUCTCODE,
  MONTH_ID,
  YEAR_ID,
  SALES,
  AVG(SALES) OVER (PARTITION BY PRODUCTCODE ORDER BY YEAR_ID,MONTH_ID ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
  AS ROLLING_3_MONTHS 




In [0]:
### yearly sales performance for each territory

In [0]:
%sql
SELECT 
  YEAR_ID,
  TERRITORY,
  SUM(SALES) AS TOTAL_SALES,
  AVG(SUM(SALES)) OVER (PARTITION BY TERRITORY ORDER BY YEAR_ID ROWS BETWEEN UNBOUNDED PRECEDING AND
  CURRENT ROW) AS AVG_SALES,
  MAX(SUM(SALES)) OVER (PARTITION BY TERRITORY ORDER BY YEAR_ID ROWS BETWEEN UNBOUNDED PRECEDING AND
  CURRENT ROW) AS MAX_SALES,

FROM productsales
GROUP BY YEAR_ID,TERRITORY
ORDER BY TERRITORY,YEAR_ID 

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [UNRESOLVED_COLUMN.WITHOUT_SUGGESTION] A column, variable, or function parameter with name `YEAR_ID` cannot be resolved.  SQLSTATE: 42703; line 2 pos 2;
'Sort ['TERRITORY ASC NULLS FIRST, 'YEAR_ID ASC NULLS FIRST], true
+- 'Aggregate ['YEAR_ID, 'TERRITORY], ['YEAR_ID, 'TERRITORY, 'SUM('SALES) AS TOTAL_SALES#7023, 'AVG('SUM('SALES)) windowspecdefinition('TERRITORY, 'YEAR_ID ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS AVG_SALES#7024, 'MAX('SUM('SALES)) windowspecdefinition('TERRITORY, 'YEAR_ID ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS MAX_SALES#7025, 'FROM AS productsales#7026]
   +- OneRowRelation

	at org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedAttributeError(QueryCompilationErrors.scala:334)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.org$apache$spark$sql$catalyst$analysis$CheckAnalysis$$failUnresolvedAttr

In [0]:
%sql
SELECT 
  YEAR_ID,
  TERRITORY,
  SUM(SALES) AS TOTAL_SALES,
  AVG(SUM(SALES)) OVER (
    PARTITION BY TERRITORY 
    ORDER BY YEAR_ID 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS AVG_SALES,
  MAX(SUM(SALES)) OVER (
    PARTITION BY TERRITORY 
    ORDER BY YEAR_ID 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS MAX_SALES
FROM 
  productsales
GROUP BY 
  YEAR_ID, TERRITORY
ORDER BY 
  TERRITORY, YEAR_ID;


YEAR_ID,TERRITORY,TOTAL_SALES,AVG_SALES,MAX_SALES
2003,APAC,253134.45000000004,253134.45000000004,253134.45000000004
2004,APAC,345307.8299999998,299221.1399999999,345307.8299999998
2005,APAC,147679.55000000002,248707.2766666666,345307.8299999998
2003,EMEA,1660314.5300000005,1660314.5300000005,1660314.5300000005
2004,EMEA,2389128.889999999,2024721.71,2389128.889999999
2005,EMEA,929828.9899999998,1659757.47,2389128.889999999
2003,Japan,243773.18,243773.18,243773.18
2004,Japan,168479.09999999992,206126.13999999996,243773.18
2005,Japan,42920.939999999995,151724.40666666665,243773.18
2003,NA,1359757.3799999997,1359757.3799999997,1359757.3799999997


In [0]:
### monthly totalorders performance of each productline - sum,median,variance,average,max


In [0]:
%sql

SELECT
  MONTH_ID,
  PRODUCTLINE,
  SUM(QUANTITYORDERED) AS TOTAL_ORDERS,
  AVG(SUM(QUANTITYORDERED)) OVER (PARTITION BY PRODUCTLINE ORDER BY MONTH_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS AVG_ORDERS,
  MAX(SUM(QUANTITYORDERED)) OVER (PARTITION BY PRODUCTLINE ORDER BY MONTH_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS MAX_ORDERS,
percentile_approx(SUM(QUANTITYORDERED),0.5)  OVER (PARTITION BY PRODUCTLINE ORDER BY MONTH_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS MEDIAN,
variance(SUM(QUANTITYORDERED)) OVER (PARTITION BY PRODUCTLINE ORDER BY MONTH_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS VARIANCE

FROM productsales
GROUP BY 
  MONTH_ID, PRODUCTLINE
ORDER BY 
  PRODUCTLINE, MONTH_ID;



MONTH_ID,PRODUCTLINE,TOTAL_ORDERS,AVG_ORDERS,MAX_ORDERS,MEDIAN,VARIANCE
1,Classic Cars,2740,2740.0,2740,2740,null
2,Classic Cars,2586,2663.0,2740,2586,11858.0
3,Classic Cars,2458,2594.6666666666665,2740,2586,19937.333333333336
4,Classic Cars,2310,2523.5,2740,2458,33550.333333333314
5,Classic Cars,3057,2630.2,3057,2586,82087.19999999998
6,Classic Cars,1080,2371.8333333333335,3057,2458,466189.7666666666
7,Classic Cars,2028,2322.714285714286,3057,2458,405380.23809523805
8,Classic Cars,2224,2310.375,3057,2310,348686.8392857142
9,Classic Cars,1938,2269.0,3057,2310,320507.9999999999
10,Classic Cars,3818,2423.9,3818,2310,524836.1000000001


Databricks visualization. Run in Databricks to view.

### 13. Identify Consecutive Quarters of sales decline for each product

In [0]:
%sql
WITH SalesLag AS (
  SELECT 
    QTR_ID,
    PRODUCTCODE,
    SUM(SALES) AS TOTAL_SALES,
    LAG(SUM(SALES),1) OVER(PARTITION BY PRODUCTCODE ORDER BY QTR_ID) AS PREVIOUS_QUARTER_SALES,
    LAG(SUM(SALES),2) OVER(PARTITION BY PRODUCTCODE ORDER BY QTR_ID) AS TWO_QUARTER_AGO_SALES
  FROM productsales
  GROUP BY QTR_ID,PRODUCTCODE
)
SELECT
  QTR_ID,
  PRODUCTCODE,
  TOTAL_SALES,
  PREVIOUS_QUARTER_SALES,
  TWO_QUARTER_AGO_SALES
FROM SalesLag
WHERE TOTAL_SALES < PREVIOUS_QUARTER_SALES AND PREVIOUS_QUARTER_SALES < TWO_QUARTER_AGO_SALES
ORDER BY QTR_ID,PRODUCTCODE

QTR_ID,PRODUCTCODE,TOTAL_SALES,PREVIOUS_QUARTER_SALES,TWO_QUARTER_AGO_SALES
3,S10_4757,15525.760000000002,28118.000000000004,30667.030000000002
3,S10_4962,21557.940000000002,26600.789999999997,29109.129999999997
3,S12_1108,34228.520000000004,36597.58,42518.53999999999
3,S12_1666,22350.920000000002,29229.57,29907.78
3,S12_3148,23269.35,25896.08,30764.62
3,S12_4675,16340.59,21892.25,25090.4
3,S18_1097,20474.579999999998,23156.0,28432.050000000003
3,S18_1749,14919.2,29819.699999999997,31003.16
3,S18_1889,14673.890000000001,16225.439999999999,17179.46
3,S18_2248,4799.64,10000.23,17959.859999999997


Databricks visualization. Run in Databricks to view.

In [0]:
### identity top 10% of products by sales sin each quarter

In [0]:
%sql
WITH SalesRank AS (
  SELECT
    QTR_ID,
    PRODUCTCODE,
    SALES,
    percent_rank() OVER (PARTITION BY QTR_ID ORDER BY SALES) AS SALES_PERCENT_RANK
  FROM productsales
)
SELECT
    QTR_ID,PRODUCTCODE,SALES FROM SalesRank
    WHERE SALES_PERCENT_RANK>0.1
    ORDER BY QTR_ID,SALES_PERCENT_RANK

QTR_ID,PRODUCTCODE,SALES
1,S12_3990,1627.92
1,S18_4721,1629.04
1,S24_3420,1630.6
1,S18_4522,1640.24
1,S700_1691,1649.55
1,S24_3969,1654.56
1,S24_4278,1657.76
1,S32_2509,1666.7
1,S18_2432,1667.4
1,S700_2834,1668.25


In [0]:
### CATEGORIZE sales percent as high,medium or low lessthan 25 low (ntile(100))

In [0]:
%sql
WITH SalesPercentile AS (
  SELECT 
    QTR_ID,
    PRODUCTCODE,
    SALES,
    NTILE(100) OVER(PARTITION BY QTR_ID ORDER BY SALES DESC) AS PERCENTILE
  FROM productsales 
)
SELECT
  QTR_ID,
  PRODUCTCODE,
  SALES,
  CASE
      WHEN PERCENTILE <=25  THEN "Low"
      WHEN PERCENTILE BETWEEN 26 AND 75  THEN "MEDIUM"
      ELSE "HIGH"
  END AS PERFORMANCE_CATEGORY
FROM SalesPercentile
ORDER BY QTR_ID,PRODUCTCODE

QTR_ID,PRODUCTCODE,SALES,PERFORMANCE_CATEGORY
1,S10_1678,733.11,HIGH
1,S10_1678,3207.12,MEDIUM
1,S10_1678,3965.66,MEDIUM
1,S10_1678,4708.44,Low
1,S10_1678,2871.0,MEDIUM
1,S10_1949,2416.56,MEDIUM
1,S10_1949,6075.3,Low
1,S10_1949,6463.23,Low
1,S10_1949,4514.92,Low
1,S10_1949,7209.11,Low


In [0]:
### month with highest sales in each year

In [0]:
%sql
select year_id,month_id,total_sales,rank
from (select year_id,month_id,total_sales,dense_rank() over(partition by year_id order by total_sales desc)
as rank from (
  select year_id,month_id,sum(sales) as total_sales from productsales group by year_id,month_id) as aggregatedsales) as rankedsales
where rank=1 order by year_id,month_id

year_id,month_id,total_sales,rank
2003,11,1029837.6600000001,1
2004,11,1089048.0100000005,1
2005,5,457861.05999999965,1
